### HuggingFace Semantic Similarity Estimation
What are we Doing? 
This is essentially a fact finding experiment. 

We want to estimate the possibility of using the current corpus as part of a model to evaluate potential future articles

<div>
   <img src="../../images/SemanticEstimation.png" height="374"; width="374">
</div>

In [1]:
import sys
import os
# LIBRARY_PATH = '/home/deleidos/Notebooks/Newspaper/SemanticEstimation'
LIBRARY_PATH = "/home/deleidos/Notebooks/lib"
module_path = os.path.abspath(os.path.join(LIBRARY_PATH))
if module_path not in sys.path:
    sys.path.append(module_path)
print (f"sys.path : {sys.path}") 


sys.path : ['/home/deleidos/Notebooks/Newspaper/SemanticEstimation', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/home/deleidos/.ipython', '/home/deleidos/Notebooks/lib']


In [2]:
#!pip install sentence_transformers
import torch

import os, csv
from csv import writer, DictWriter
from os import listdir
import os.path as osp
from os.path import isfile, isdir, join

import time
import datetime
from datetime import datetime as DT
import json
import random
# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

import nltk
# nltk.download('punkt')
# nltk.download('brown')
# Locals 
from pathlib import Path 
import os, sys 
import glob

from IPython.display import Image
from IPython.core.display import HTML 
from sentence_transformers import SentenceTransformer, util

# nltk.download('punkt')
###
module_path = str(Path.cwd().parents[0] / "")
if module_path not in sys.path:
    sys.path.append(module_path)   
from utils import *
import random
from timeit import default_timer as timer
import time
import datetime
from datetime import datetime as DT
from os import listdir
import os.path as osp
from os.path import isfile, join

# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

import pandas as pd
import numpy as np
from dynamoDButils import *
# regex
import re
p = re.compile('[a-z]+')
p
re.compile('[a-z]+')
## LOCAL 
CSVPath = "/home/deleidos/Notebooks/CSVData"
JSONPath = "/home/deleidos/Notebooks/JSONData"
from APIutils import *

###
def complete(state="complete"):
    print(f"\nCell {state} @ {(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))}")
complete(state='imports done')


/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



Cell imports done @ 2023-03-03 03:48:20


[nltk_data] Downloading package punkt to /home/deleidos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
%%time

RESOURCE = 'dynamodb'
TABLE = "Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev"
TABLE = "Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev"
REGION = "us-east-1"

os.environ['AWS_DEFAULT_REGION']= region
print (f"env : {os.getenv('AWS_DEFAULT_REGION')}")
print('\n\nconnecting ...')
exists = table_exists(RESOURCE, REGION, TABLE)
complete(state= "DB read")

env : us-east-1


connecting ...
Table {table.name} doesn't exist.

Cell DB read @ 2023-03-03 03:48:32
CPU times: user 86.6 ms, sys: 14 ms, total: 101 ms
Wall time: 121 ms


In [ ]:
resp = extendedScanArticlesWithKey(RESOURCE,TABLE, columns="title,summary", keyword="url")
print (f"Resp[3400] : {resp[3400]}")

#### Notes:
The format of each returned item resembles:

    'summary': {'S': 
    "Stock futures are trading up slightly Monday night as investors tried to shake the previous session's selloff.S&P 500 
    futures were fractionally higher while Nasdaq-100 futures fell marginally.Monday's close marked a negative start to the 
    trading week.Market observers are still largely expecting a 50 basis point increase to interest rates at the Fed's 
    December meeting.Investors will look ahead to data Tuesday morning on international trade for insight into the strength 
    of the U.S. and global economy."}, 
    'url': {'S': 'https://www.cnbc.com/2022/12/05/stock-market-futures-open-to-close-news.html'}, 
    'title': {'S': 'Stock futures are up slightly as fears mount over upcoming interest rate hikes'}



In [ ]:
# for item in resp[:2]:  # print  first 10 articles 
#     print (f"URL: {item['url']['S']}\nTitle: {item['title']['S']}:\nSummary: {item['summary']['S']}", end="\n\n")
# # TODO:
# # clean up the return s/t it does not have stuff like urls or (money, per centage)

#####  Write the return items into a JSON  file s/t we can later read back in and use 

In [ ]:
# Serializing json
indx = 0
jsndict = {}
for item in resp:
    jsndict[str(indx)]= {
        "url" : item['url']['S'],
        "title" : item['title']['S'],
        "summary": item['summary']['S']
    }
    indx += 1
    
json_object = json.dumps(jsndict, indent=4)
jsonfile = join(JSONPATH, "semanticSimArticlesME.json")


with open(jsonfile, "w") as outfile:
     outfile.write(json_object)
complete(state="JSON created")

In [ ]:
jsonfiles = [f for f in listdir(JSONPATH) if isfile(join(JSONPATH, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsnWD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsnWD.observe(on_change)
display(jsnWD)

In [ ]:
jsnf = jsnWD.value
print (f"{jsnf=} ")

In [ ]:
jsnfile = osp.join (JSONPATH,jsnf)
df = pd.read_json(jsnfile)
xposedf = df.transpose() # pivot the matrix

In [ ]:
xposedf

In [ ]:
summaries = xposedf.summary.to_list()
# print(f"Summaries-first: {summaries[0]}")

In [ ]:
%%time
model = SentenceTransformer('all-MiniLM-L6-v2')
complete(state="SentenceTransformer")

In [ ]:
%%time
corpus = summaries
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)
complete(state="model.encode")

In [ ]:
#START HERE
### read in the target summaries to evaluate against
tgtfiles = [f for f in listdir(MODELPATH) if isfile(join(MODELPATH, f))]
tgtfiles.sort()
tgtfiles = [x for x in tgtfiles if (os.path.splitext(x)[1][1:]).lower() == "tgt"]
tgtWD = widgets.Select(
    options=tgtfiles,
    value=tgtfiles[0],
    description='Choose File:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

tgtWD.observe(on_change)
display(tgtWD)


In [ ]:
%%time
tgtf =  tgtWD.value
print (f"{tgtf=} ")

tgtfile = osp.join (MODELPATH,tgtf)
df = pd.read_csv(tgtfile, sep="^")
# df = df['ABSTRACTS']
iter = 0
for index, row in df.iterrows():
#     print(row['c1'], row['c2'])
    iter += 1
    row['SOURCE'] = row['SOURCE'] + str(iter)
df

In [ ]:
### convert summaries to a list
queries = df["SUMMARY"].tolist()
print(queries[0])
print(len(queries))

In [ ]:
%%time
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries[:1]:
    query_embedding = model.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1],):
        print(corpus[idx], "(Score: {:.4f})".format(score))
            

In [ ]:
%%time
# Make dictionary of lists of the scores for each entry
hf_score = {'hf1': [], 'hf2': [], 'hf3': [], 'hf4': [],'hf5': []}

# Find the closest 5 entries of the corpus for each query entry based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    
    top_r = torch.topk(cos_scores, k=top_k)
    scores = top_results[0].cpu().data.numpy()
    indices = top_results[1].cpu().data.numpy()

    elt = 0
    for score, idx in zip(scores, indices):
        score = round(score, 3) # move to cpu memory
        elt += 1
        hf_score['hf'+str(elt)].append(score)

complete(state="done")

In [ ]:
# Make a df of the HF scores collected
hf_df = pd.DataFrame(hf_score)
hf_df.head(10)

In [ ]:
#Make a new master df with the source, summary, and corresponding hf scores
mixed_df = pd.concat([df, hf_df], axis=1, join='inner')
mixed_df.head(10)

In [ ]:
def removeOutliers(test_list):
    q1 = np.percentile(test_list, 25)
    q3 = np.percentile(test_list, 75)
    iqr = (q3 - q1) * 1.5
    q_set = (q1 - iqr, q3 + iqr)

    result_list =[]

    for num in test_list:
        if num >= q_set[0] and num <= q_set[1]:
            result_list.append(num)

    return result_list

In [ ]:
def calculateScoreAvg(df):
    score_avg = []
    
    for i in range(len(df)):
        row_list = df.loc[i, ['hf1', 'hf2', 'hf3', 'hf4', 'hf5']].values.flatten().tolist()
        test_list = list(map(float, row_list))
        
        new_list = removeOutliers(test_list)
        weighted_score = np.mean(new_list)
#         print(f"Weighted score of row {i}: {weighted_score}")
        
        score_avg.append(weighted_score)
        i +=1
        
    return score_avg

In [ ]:
hf_avg = calculateScoreAvg(mixed_df)

mixed_df['hfAvg'] = hf_avg
mixed_df.head(10)

In [ ]:
mixed_df.rename(columns = {'SOURCE':'source', 'SUMMARY':'summary'}, inplace = True)
mixed_df.head(10)

In [ ]:
jsonfile = join(JSONPath, "semanticSimArticlesTestingME.json")
print(jsonfile)

mixed_df.to_json(jsonfile)
complete(state="updates written: ")

In [ ]:
# #Make a list of tuples with your data and then create a DataFrame with it:
# site = 'mixed-entries'
# jsndict = {}
# jsndict[site] =  {"summary" : summary}
# entries = {"scores" : {}}

# top_r = torch.topk(cos_scores, k=top_k)
# scores = top_results[0].cpu().data.numpy()
# indices = top_results[1].cpu().data.numpy()

# elt = 0
# for score, idx in zip(scores, indices):
#     score = round(score, 3) # move to cpu memory
#     entry = {
#         "corpusindex" : str(idx),
#         "score" : str(score),
#         "corpusURL" : xposedf.iloc[idx].url,
#         "corpusSummary" : xposedf.iloc[idx].summary,
#     }
#     elt += 1
#     entries['scores'].update({'HF'+str(elt) : entry})
    
# jsndict[site].update(entries)

# # print (f"results: {jsndict}")

# complete(state="done")

In [ ]:
# jsonfile = join(JSONPath, "semanticSimArticlesTestingME.json")
# # read this in as an array
# element = {}
# try:
#     with open(jsonfile, "r") as infile:
#         element = json.load(infile)
# except FileNotFoundError as e:
#     print (f"Error : {e}. Creating a new list")
    

# element.update(jsndict)

# print (len(element.keys()))
# json_object = json.dumps(element, indent=4)

# with open(jsonfile, "w") as outfile:
#      outfile.write(json_object)
